In [1]:
import torch
from joetorch.nn.modules import ConvSelfAttentionBlock, ConvCrossAttentionBlock

In [4]:
sa = ConvSelfAttentionBlock(16, 1)

x = torch.randn(128, 16, 8, 8)
y = sa(x)
print(y.shape)

ca = ConvCrossAttentionBlock(16, 1)

z = torch.randn(128, 16, 8, 8)
e = torch.randn(128, 16, 2, 2)
w = ca(z, e)
print(w.shape)

torch.Size([128, 16, 8, 8])
torch.Size([128, 16, 8, 8])
